# 3. プロンプトエンジニアリング


In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

#### 【注意】既知のエラーについて

openai パッケージが依存する httpx のアップデートにより、`openai==1.40.6` を使用する箇所で `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'` というエラーが発生するようになりました。

このエラーは、`!pip install httpx==0.27.2` のように、httpx の特定バージョンをインストールすることで回避することができます。

なお、Google Colab で一度上記のエラーに遭遇したあとで `!pip install httpx==0.27.2` のようにパッケージをインストールし直した場合、以下のどちらかの操作を実施する必要があります。

- Google Colab の「ランタイム」から「セッションを再起動する」を実行する
- 「ランタイムを接続解除して削除」を実行してパッケージのインストールからやり直す


In [2]:
!pip install openai==1.40.6 httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: openai
    Found existing installation: openai 1.91.0
    Uninstalling openai-1.91.0:
      Successfully uninstalled openai-1.91.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.21.1 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


## 3.2. プロンプトエンジニアリングとは


In [3]:
from openai import OpenAI  # OpenAIのライブラリを使うよ（ChatGPTと話す準備をする）

client = OpenAI()  # ChatGPTとつながるための電話を作るよ

response = client.chat.completions.create(  # ChatGPTに質問して返事をもらうための命令だよ
    model="gpt-4o-mini",  # gpt-4o-miniという軽くて速いAIを使うよ
    messages=[
        {"role": "user", "content": "プロンプトエンジニアリングとは"},  # ユーザーからの質問「プロンプトエンジニアリングってなに？」
    ],
)

print(response.choices[0].message.content)  # AIからの返事の文章だけを取り出して表示するよ


プロンプトエンジニアリング（Prompt Engineering）とは、特に大規模な言語モデル（LLM）やAIシステムとの対話において、最適な応答を引き出すための入力（プロンプト）を設計・調整する技術や手法のことを指します。AIに与えるプロンプトの内容や形式を工夫することで、より正確で適切な出力を得ることが目的です。

プロンプトエンジニアリングの主なポイントは以下の通りです：

1. **プロンプトの設計**: より良い結果を得るためには、質問の仕方や情報の提示方法を工夫する必要があります。たとえば、具体的な指示や例を提供することが重要です。

2. **実験と調整**: 異なるプロンプトを試し、出力を比較することで、最も効果的なプロンプトを見つけるプロセスが含まれます。

3. **コンテキストの活用**: モデルが適切に反応できるように、必要なコンテキスト情報を含めることが重要です。

4. **ユーザーの意図を反映させる**: ユーザーが求める情報や結果を明確にするため、プロンプトに対する意図を明示することが求められます。

この技術は、チャットボットの開発、コンテンツ生成、データの要約、自然言語処理のタスクにおいて非常に重要な役割を果たしています。プロンプトエンジニアリングを適切に行うことで、AIの能力を最大限に引き出し、実用的かつ効果的な結果を得ることが可能になります。


In [4]:
response = client.chat.completions.create(  # ChatGPTに質問して返事をもらう命令だよ
    model="gpt-4o-mini",  # 軽くて速いGPT-4o-miniというAIを使うよ
    messages=[  # ChatGPTに送るメッセージのリストだよ
        {"role": "system", "content": "質問に100文字程度で答えてください。"},  # 最初にAIにルールを伝えるよ「100文字くらいで答えてね」
        {"role": "user", "content": "プロンプトエンジニアリングとは"},  # ユーザーからの質問だよ「プロンプトエンジニアリングってなに？」
    ],
)
print(response.choices[0].message.content)  # AIから返ってきた答えの“中身の文章”だけを画面に表示するよ


プロンプトエンジニアリングとは、AIや機械学習モデルに対して効果的に指示を与えるための技術や手法を指します。適切な質問や命令を設計し、モデルから望ましい応答を引き出すことを目的としています。主に自然言語処理の分野で活用されています。


## 3.3. プロンプトの構成要素の基本


### プロンプトのテンプレート化


In [5]:
prompt = '''\  # ChatGPTに送るメッセージのひな型（テンプレート）を作るよ
以下の料理のレシピを考えてください。

料理名: """
{dish}
"""
'''  # {dish} のところに、あとで作りたい料理の名前を入れるよ


def generate_recipe(dish: str) -> str:  # 「料理名」をもとにレシピを作る関数を作ってるよ
    response = client.chat.completions.create(  # ChatGPTに質問して、レシピを作ってもらう命令だよ
        model="gpt-4o-mini",  # 軽くて速いAI（GPT-4o-mini）を使うよ
        messages=[
            {"role": "user", "content": prompt.format(dish=dish)},  # テンプレートの中に「料理名」を入れて質問を送ってるよ
        ],
    )
    return response.choices[0].message.content  # ChatGPTが返してくれたレシピの本文だけを返してるよ


recipe = generate_recipe("カレー")  # 「カレー」という料理のレシピを作ってもらうよ
print(recipe)  # 出てきたレシピを画面に表示するよ


もちろん、カレーのレシピをご紹介します。以下は基本的なカレーの作り方です。

---

### カレーのレシピ

#### 材料（4人分）
- 鶏肉（または牛肉、豚肉）：500g
- 玉ねぎ：2個
- にんじん：1本
- じゃがいも：2個
- カレールー：1箱（約100g）
- サラダ油：大さじ2
- 水：800ml
- 塩：適量
- 胡椒：適量
- お好みでガーリック、しょうが：各1かけ（みじん切り）

#### 作り方

1. **材料の下ごしらえ**
   - 鶏肉（または選んだ肉）を一口大に切ります。
   - 玉ねぎは薄切り、にんじんとじゃがいもは一口大の乱切りにします。

2. **炒める**
   - 鍋にサラダ油を熱し、玉ねぎを透き通るまで炒めます。
   - お好みでガーリックやしょうがを加え、香りが立つまでさらに炒めます。

3. **肉を加える**
   - 鶏肉を鍋に加え、表面が白くなるまで炒めます。

4. **野菜を加える**
   - にんじんとじゃがいもを加え、全体を軽く炒めます。

5. **水を加える**
   - 水を加え、強火で煮立たせます。その後、中火にしてアクを取りながら約15分煮ます。

6. **カレールーを加える**
   - カレールーを割り入れ、よくかき混ぜて溶かします。さらに中火で10〜15分煮込み、好みの濃度になるまで煮続けます。

7. **味を調える**
   - 塩と胡椒で味を整えます。必要に応じてさらに煮込みます。

8. **盛り付け**
   - ご飯と一緒に皿に盛り付け、お好みで福神漬けやらっきょうを添えて完成です。

---

### ポイント
- お好みで他の野菜（ピーマン、ナスなど）やスパイスを使って自分好みにアレンジできます。
- 水の量や煮込む時間によって濃度を調整することができます。

ぜひお試しください！


In [6]:
prompt = """\  # レシピをお願いするための文章テンプレートを作ってるよ（今は使われてないけど）
ユーザーが入力した料理のレシピを考えてください。

料理名: '''
{dish}
'''
"""  # {dish} の部分に料理名を入れる形だよ（でもこの変数は下の関数では使われてないよ）


def generate_recipe(dish: str) -> str:  # 「料理名」を入力としてレシピを作る関数だよ
    response = client.chat.completions.create(  # ChatGPTに質問を送って返事（レシピ）をもらうよ
        model="gpt-4o-mini",  # 軽くて速いGPT-4o-miniというAIを使うよ
        messages=[
            {"role": "system", "content": "ユーザーが入力した料理のレシピを考えてください。"},  # AIに「ルール（役割）」を教えてるよ
            {"role": "user", "content": f"{dish}"},  # ユーザーの入力として料理名（例：カレー）を送ってるよ
        ],
    )
    return response.choices[0].message.content  # ChatGPTが返してくれたレシピの内容だけを取り出して返してるよ


recipe = generate_recipe("カレー")  # 「カレー」という料理名でレシピを考えてもらうよ
print(recipe)  # 出てきたレシピを画面に表示するよ


カレーのレシピを考えてみましょう！ここでは、一般的なチキンカレーのレシピをご紹介します。

### チキンカレーのレシピ

#### 材料（4人分）
- 鶏もも肉：400g
- 玉ねぎ：2個
- にんにく：2片
- 生姜：1片
- トマト：1個（またはトマト缶1/2缶）
- カレールー：適量（お好みで）
- サラダ油：大さじ2
- 水：500ml
- 塩：適量
- コショウ：適量
- お好みでスパイス（クミン、ターメリック、ガーリックパウダー、チリパウダーなど）
- パセリ（飾り用）

#### 作り方
1. **下準備**
   - 鶏もも肉は一口大に切り、塩とコショウで下味をつけておく。
   - 玉ねぎ、にんにく、生姜はそれぞれみじん切りにする。
   - トマトはざく切りにする。

2. **炒める**
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で透明になるまで炒める。
   - にんにくと生姜を加え、香りが立つまでさらに炒める。

3. **鶏肉を加える**
   - 鶏もも肉を加え、表面が白くなるまで炒める。

4. **トマトと水を加える**
   - トマトを加えてさっと炒めた後、水を入れ、煮立たせる。

5. **煮込む**
   - アクを取り、弱火にして15-20分ほど煮込む。

6. **カレールーを加える**
   - カレールーを入れ、よく混ぜて溶かす。お好みでスパイスを加えて調整する。

7. **味を調える**
   - 塩とコショウで味を調整し、さらに10分ほど煮込む。

8. **盛り付け**
   - ご飯と一緒に皿に盛り付け、パセリを振りかけて完成！

### おすすめのサイドディッシュ
- ナンやライス、ピクルス、サラダなどが合います。

### 確認ポイント
- お好みの辛さに合わせてカレールーやスパイスの量を調整してください。
- 野菜（じゃがいも、にんじん、ピーマンなど）を加えることで、よりボリュームアップができます。

お楽しみください！


### 出力形式を指定する


In [8]:
system_prompt = """\
ユーザーが入力した料理のレシピを考えてください。

出力は以下のJSON形式にしてください。

```
{
  "材料": ["材料1", "材料2"],
  "手順": ["手順1", "手順2"]
}
```
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "カレー"},
    ],
)
print(response.choices[0].message.content)

{
  "材料": ["米", "カレールー", "鶏肉", "玉ねぎ", "じゃがいも", "人参", "水"],
  "手順": [
    "米を洗い、炊飯器で炊く。",
    "鶏肉、玉ねぎ、じゃがいも、人参を一口大に切る。",
    "鍋に油を熱し、玉ねぎを炒める。",
    "玉ねぎが透明になったら鶏肉を加え、表面が白くなるまで炒める。",
    "じゃがいもと人参を加え、さらに炒める。",
    "水を加え、沸騰したらアクを取り、蓋をして弱火で煮る。",
    "野菜が柔らかくなったら、カレールーを加え、よく溶かす。",
    "中火で5分ほど煮込み、全体がなじんだら火を止める。",
    "炊き上がったご飯を皿に盛り、カレーをかけて完成。"
  ]
}


## 3.4. プロンプトエンジニアリングの定番の手法


### Zero-shot プロンプティング


In [9]:
response = client.chat.completions.create(  # ChatGPTにメッセージを送って返事をもらうよ
    model="gpt-4o-mini",  # 軽くて速いGPT-4o-miniというAIを使ってるよ
    messages=[  # ChatGPTに送るメッセージのやりとりをここに書くよ
        {
            "role": "system",  # これはChatGPTに「ルール（役割）」を伝える部分だよ
            "content": "入力をポジティブ・ネガティブ・中立のどれかに分類してください。",  # 感情を3つにわけてね！とお願いしてるよ
        },
        {
            "role": "user",  # これはユーザーからの発言（質問や意見）だよ
            "content": "ChatGPTはとても便利だ",  # ユーザーが「ChatGPTは便利！」と発言してるよ
        },
    ],
)

print(response.choices[0].message.content)  # ChatGPTの答え（感情分類の結果）を画面に表示するよ


ポジティブ


### Few-shot プロンプティング


In [10]:
response = client.chat.completions.create(  # ChatGPTに質問を送って、返事をもらうよ
    model="gpt-4o-mini",  # 軽くて速いGPT-4o-miniというAIを使うよ
    messages=[  # ChatGPTとやりとりするメッセージをリストにして渡すよ
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},  # AIに「これがAIに関係してるかどうか教えてね」とお願いしてるよ
        {"role": "user", "content": "ChatGPTはとても便利だ"},  # ユーザーが「ChatGPTは便利」と言ってるよ（これはAIの話だよね）
    ],
)

print(response.choices[0].message.content)  # ChatGPTの返事（AIに関係あるかどうか）を画面に表示するよ


はい、ChatGPTは多くのユーザーにとって役立つツールです。質問に答えたり、情報を提供したり、アイデアを考える手助けをすることで、さまざまな用途に応じたサポートを提供します。他にも知りたいことがあれば、お気軽にどうぞ！


In [11]:
response = client.chat.completions.create(  # ChatGPTにメッセージを送って返事をもらうよ
    model="gpt-4o-mini",  # 軽くて速いGPT-4o-miniというAIを使うよ
    messages=[  # これまでの会話を全部渡して、続きを考えてもらうよ
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},  # 最初に「AIに関係あるか判断してね」とルールを伝えてるよ
        {"role": "user", "content": "AIの進化はすごい"},  # 最初の質問「これはAIの話？」
        {"role": "assistant", "content": "true"},  # ChatGPTが「うん、AIの話だよ」と答えた
        {"role": "user", "content": "今日は良い天気だ"},  # 次の質問「これはAIの話？」
        {"role": "assistant", "content": "false"},  # ChatGPTが「いや、これはAI関係ないね」と答えた
        {"role": "user", "content": "ChatGPTはとても便利だ"},  # 新しい質問「これはAIの話？」
    ],
)

print(response.choices[0].message.content)  # ChatGPTが考えた返事（true か false）を画面に出すよ


true


### （コラム）Few-shot プロンプティングのその他の形式


In [12]:
prompt = """\  # ChatGPTに渡す質問とヒントのセットを作ってるよ
入力がAIに関係するか回答してください。

Q: AIの進化はすごい
A: true
Q: 今日は良い天気だ
A: false
Q: ChatGPTはとても便利だ
A:
"""  # これは「これはAIの話？ そうじゃない？」というクイズの見本と質問のセットだよ


response = client.completions.create(  # ChatGPTに質問を送って返事をもらうよ
    model="gpt-3.5-turbo-instruct",  # 命令文（instruct）に強いモデルを使ってるよ
    prompt=prompt,  # さっき作ったクイズ形式のテキストをそのまま渡してるよ
)

print(response.choices[0].text)  # ChatGPTの返事（true または false）を画面に表示するよ


true


In [13]:
response = client.chat.completions.create(  # ChatGPTに質問を送って、返事をもらうよ
    model="gpt-4o-mini",  # 軽くて速いGPT-4o-miniというAIを使ってるよ
    messages=[  # ChatGPTに渡す会話の流れをここに書いてるよ
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},  # 最初にルール「AIに関係あるかどうか教えてね！」と伝えるよ

        {"role": "system", "name": "example_user", "content": "AIの進化はすごい"},  # 例としての質問①「これはAIの話？」
        {"role": "system", "name": "example_assistant", "content": "true"},  # それに対する例の答え①「AIの話だからtrue」

        {"role": "system", "name": "example_user", "content": "今日は良い天気だ"},  # 例としての質問②「これはAIの話？」
        {"role": "system", "name": "example_assistant", "content": "false"},  # それに対する例の答え②「天気の話だからfalse」

        {"role": "user", "content": "ChatGPTはとても便利だ"},  # 本番の質問「これはAIの話かな？」
    ],
)

print(response.choices[0].message.content)  # ChatGPTが出した答え（trueかfalse）を画面に表示するよ


true


### Zero-shot Chain-of-Thought プロンプティング


In [14]:
response = client.chat.completions.create(  # ChatGPTに質問を送って、答えだけをもらう命令だよ
    model="gpt-4o-mini",  # 軽くて速いGPT-4o-miniというAIを使ってるよ
    messages=[
        {"role": "system", "content": "回答だけ一言で出力してください。"},  # AIに「答えだけ1つ出してね」とルールを伝えてるよ
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},  # ユーザーが計算式を送ってるよ（AIに計算してもらう）
    ],
)

print(response.choices[0].message.content)  # ChatGPTが返してきた計算結果を画面に表示するよ（余計な説明なし！）


6


In [15]:
response = client.chat.completions.create(  # ChatGPTに質問を送って、考え方つきで返してもらう命令だよ
    model="gpt-4o-mini",  # 軽くて速いGPT-4o-miniというAIを使ってるよ
    messages=[
        {"role": "system", "content": "ステップバイステップで考えてください。"},  # 「順番にゆっくり考えてね」とAIにお願いしてるよ
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},  # ユーザーが計算問題を出してるよ（優先順位があるよ！）
    ],
)

print(response.choices[0].message.content)  # ChatGPTの答えと考え方をまとめて表示するよ


この計算をステップバイステップで行いましょう。

1. **演算子の優先順位を確認**:
   - まず、掛け算と割り算 (＊, ÷) を優先的に計算し、その後に足し算と引き算 (+, -) を行います。

2. **計算を実行**:
   - 最初に、2 * 3を計算します。
     - 2 * 3 = 6
   - 次に、4 * 2を計算します。
     - 4 * 2 = 8

3. **式を置き換える**:
   - 計算結果を式に戻すと、次のようになります。
     - 10 + 6 - 8

4. **足し算を実行**:
   - 10 + 6 = 16

5. **引き算を実行**:
   - 16 - 8 = 8

したがって、最終的な答えは8です。
